<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Fabrizzio Pezzolla
- Nombre de alumno 2: Rodrigo Molina


### **Link de repositorio de GitHub:** [Link](https://github.com/frpezzolla/lab_mds.git)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('./sales.csv')

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [3]:
df['date'].shape

(7456,)

## 1 Generando un Baseline (5 puntos)

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

**1.1.** Separar los datos:

In [4]:
from sklearn import set_config
from sklearn.model_selection import train_test_split

set_config(transform_output="pandas")

# split all into train and test+dev
train_data, temp_data = train_test_split(df, test_size=0.3, random_state=42)
# split test and dev
dev_data, test_data = train_test_split(temp_data, test_size=1/3, random_state=42)

def split_X_y(data, target='quantity'):
    X_ = data.drop(columns=target)
    y_ = data[target]
    return X_,y_

X_train, y_train = split_X_y(train_data)
X_test, y_test = split_X_y(test_data)
X_dev, y_dev = split_X_y(dev_data)

**1.2.** FuncTransformer:

In [5]:
from sklearn.preprocessing import FunctionTransformer

def date_splitter(X):
    X = pd.to_datetime(X['date'], format='%d/%m/%y')
    return pd.DataFrame({
        'day': X.dt.day.astype('category'),
        'month': X.dt.month.astype('category'),
        'year': X.dt.year.astype('category')
    })

date_transformer = FunctionTransformer(date_splitter)

**1.3.** ColumnTransformer:

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

col_transformer = ColumnTransformer(
    transformers=[
        ('date', date_transformer, ['date']),
        ('onehot', OneHotEncoder(sparse_output=False), ['city', 'shop', 'brand', 'container', 'capacity']),
        ('scaler', StandardScaler(), ['lat', 'long', 'pop', 'price'])
    ],
    remainder='passthrough'
)

# col_transformer.set_output(transform='pandas')

**1.4.** DummyRegressor:

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

pipe_dummy = Pipeline([
    ('preprocessor', col_transformer),
    ('regressor', DummyRegressor(strategy='mean'))
])

**1.5.** Entrenar y obtener MAE:

In [8]:
pipe_dummy.fit(X_train, y_train)

from sklearn.metrics import mean_absolute_error

y_pred_dev = pipe_dummy.predict(X_dev)
mae_baseline = mean_absolute_error(y_dev, y_pred_dev)
mae_baseline

13298.497767341096

**1.6.** XGBRegressor:

In [9]:
from xgboost import XGBRegressor
import warnings

try:

    pipe_xgbreg = Pipeline([
        ('preprocessor', col_transformer),
        ('regressor', XGBRegressor(enable_categorical=False))
    ])

    pipe_xgbreg.fit(X = X_train, y = y_train)

except ValueError:
    warnings.warn("Can't use default arguments!")

    pipe_xgbreg = Pipeline([
    ('preprocessor', col_transformer),
    ('regressor', XGBRegressor(
        enable_categorical=True,
        random_state=42
        ))
    ])

    pipe_xgbreg.fit(X = X_train, y = y_train)
    

C:\Users\rodri\AppData\Local\Temp\ipykernel_7704\2232670368.py:14: UserWarning: Can't use default arguments!
  warnings.warn("Can't use default arguments!")


In [10]:
y_pred_dev = pipe_xgbreg.predict(X_dev)
mae_xgboost = mean_absolute_error(y_dev, y_pred_dev)
mae_xgboost

2400.27751240516

**1.7.** pickle:

In [11]:
import pickle

model_dict = {'dummy': pipe_dummy,
              'xgbreg': pipe_xgbreg}

for model_name, model_pipe in model_dict.items():
    with open(f'./pickle_1_7_{model_name}.pkl', 'wb') as pkl_file:
        pickle.dump(model_pipe, pkl_file)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [12]:
df_ = col_transformer.fit_transform(X_train)
df_.head()

,date__day,date__month,date__year,onehot__city_Athens,onehot__city_Irakleion,onehot__city_Larisa,onehot__city_Patra,onehot__city_Thessaloniki,onehot__shop_shop_1,onehot__shop_shop_2,...,onehot__container_glass,onehot__container_plastic,onehot__capacity_1.5lt,onehot__capacity_330ml,onehot__capacity_500ml,scaler__lat,scaler__long,scaler__pop,scaler__price,remainder__id
292,30,4,2012,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,1.0,1.0,0.0,0.0,-0.041439,-1.403508,-0.815231,1.659156,300
3366,28,2,2015,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,1.0,0.0,0.0,-0.201218,0.416377,1.362723,-0.594186,3416
3685,30,6,2015,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.211468,0.389617,1.362723,-0.655752,3741
2404,30,4,2014,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,-0.201218,0.416377,1.366906,-1.099033,2441
2855,30,9,2014,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,-1.800022,1.726977,-0.936680,-0.778886,2898


In [13]:
pipe_xgbreg_restricted = Pipeline([
    ('preprocessor', col_transformer),
    ('regressor', XGBRegressor(
        enable_categorical=True,
        monotone_constraints={'scaler__price':-1},
        random_state=42
        ))
])

pipe_xgbreg_restricted.fit(X_train,y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('date',
                                                  FunctionTransformer(func=<function date_splitter at 0x000001AA50C6D080>),
                                                  ['date']),
                                                 ('onehot',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['city', 'shop', 'brand',
                                                   'container', 'capacity']),
                                                 ('scaler', StandardScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price'])])),
                ('regressor',
                 XGBRegre...
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints={'scaler__price': -1},
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=42, ...))])

**2.2.** MAE:

In [14]:
y_pred_dev = pipe_xgbreg_restricted.predict(X_dev)
mae_xgboost_r = mean_absolute_error(y_dev, y_pred_dev)
mae_xgboost_r

2602.15612350863

**2.3.**
No tenía razón. El MAE crece en comparación al caso en donde se fuerza la relación monotónica. Esto quiere decir que no ayuda al modelo asumir que hay una relación inversa entre cantidad de ventas y el precio del producto.

**2.4.**

In [15]:
with open(f'./pickle_2_4_xgbreg_const.pkl', 'wb') as pkl_file:
    pickle.dump(pipe_xgbreg_restricted, pkl_file)

# 3. Optimización de Hiperparámetros con Optuna (20 puntos)

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

**3.1., 3.2., 3.3**

In [16]:
import optuna
from optuna.samplers import TPESampler
import sklearn.datasets
import sklearn.ensemble
import sklearn.svm

optuna.logging.set_verbosity(optuna.logging.DEBUG)

def objective(trial):

    pipe_param = {
        'learning_rate': trial.suggest_float('learning_rate',.001,.1),
        'n_estimators': trial.suggest_int('n_estimators',50,1000),
        'max_depth': trial.suggest_int('max_depth',3,10),
        'max_leaves': trial.suggest_int('max_leaves',0,100),
        'min_child_weight': trial.suggest_int('min_child_weight',1,5),
        'reg_alpha': trial.suggest_float('reg_alpha',0,1),
        'reg_lambda': trial.suggest_float('reg_lambda',0,1)
    }

    col_transformer = ColumnTransformer(
        transformers=[
            ('date', date_transformer, ['date']),
            ('onehot',
                OneHotEncoder(
                    sparse_output=False,
                    min_frequency = trial.suggest_float('min_frequency',.0,.1)),
                ['city', 'shop', 'brand', 'container', 'capacity']),
            ('scaler', StandardScaler(), ['lat', 'long', 'pop', 'price'])
        ],
        remainder='passthrough'
    )

    pipe_xgbreg_ = Pipeline([
        ('preprocessor', col_transformer),
        ('regressor', XGBRegressor(
            enable_categorical=True,
            random_state=42,
            **pipe_param
            ))
    ])

    # X_train, X_val, y_train, y_val = sklearn.model_selection.train_test_split(X, y, random_state=42)

    pipe_xgbreg_.fit(X_train, y_train)
    y_pred = pipe_xgbreg_.predict(X_dev)

    error = sklearn.metrics.mean_absolute_error(y_dev, y_pred)

    return error  # An objective value linked with the Trial object.

study = optuna.create_study(sampler=TPESampler())  # Create a new study.
study.optimize(objective, timeout=5*60)  # Invoke optimization of the objective function.

[I 2024-10-25 00:00:59,238] A new study created in memory with name: no-name-400f65da-627f-4f59-b12d-c0f51f9ba37f
[I 2024-10-25 00:01:00,665] Trial 0 finished with value: 2109.6047680942424 and parameters: {'learning_rate': 0.07300853010389087, 'n_estimators': 573, 'max_depth': 5, 'max_leaves': 65, 'min_child_weight': 4, 'reg_alpha': 0.7530611989562601, 'reg_lambda': 0.07198627501001542, 'min_frequency': 0.0791332886704157}. Best is trial 0 with value: 2109.6047680942424.
[I 2024-10-25 00:01:00,998] Trial 1 finished with value: 9515.607919653337 and parameters: {'learning_rate': 0.004702099603579871, 'n_estimators': 142, 'max_depth': 8, 'max_leaves': 17, 'min_child_weight': 1, 'reg_alpha': 0.9810716857110336, 'reg_lambda': 0.6148643673841265, 'min_frequency': 0.08096051282910686}. Best is trial 0 with value: 2109.6047680942424.
[I 2024-10-25 00:01:02,533] Trial 2 finished with value: 3342.075292903253 and parameters: {'learning_rate': 0.005696749471875801, 'n_estimators': 815, 'max_dep

In [17]:
study.set_user_attr('best_trial',study.best_trial)

best_params = study.best_params

col_transformer = ColumnTransformer(
    transformers=[
        ('date', date_transformer, ['date']),
        ('onehot',
            OneHotEncoder(
                sparse_output=False,
                min_frequency = best_params['min_frequency']),
            ['city', 'shop', 'brand', 'container', 'capacity']),
        ('scaler', StandardScaler(), ['lat', 'long', 'pop', 'price'])
    ],
    remainder='passthrough'
)

del best_params['min_frequency']

pipe_xgbreg_ = Pipeline([
    ('preprocessor', col_transformer),
    ('regressor', XGBRegressor(
        enable_categorical=True,
        random_state=42,
        device='cuda',
        **best_params
        ))
])

study.set_user_attr('best_pipeline', pipe_xgbreg_)

In [18]:
study.best_value

1970.6179534891642

El MAE es más bajo. Esto se debe a que el algoritmo de optuna busca la mejor combinación de parámetros respecto al MAE según la predicción del modelo seleccionado. Optuna ha encontrado el mejor punto medio (dentro de las pruebas) entre complejidad del árbol y generalización al conjunto de validación.

**3.4.**

In [19]:
study.best_params

{'learning_rate': 0.061495828095354074,
 'n_estimators': 926,
 'max_depth': 8,
 'max_leaves': 68,
 'min_child_weight': 5,
 'reg_alpha': 0.0028755461558962053,
 'reg_lambda': 0.12085974287977863,
 'min_frequency': 0.012681898035110887}

- `learning_rate` o *shrinkage* cuantifica la influencia de cada árbol sobre cada predicción. Modelos más conservativos tendrán *shrinkage* más pequeño para evitar el sobreajuste. Se buscaron valores entre $0.0$ y $1.0$, lo que se condice con la [documentación](https://xgboost.readthedocs.io/en/latest/parameter.html).
- `n_estimators` se refiere a la cantidad de rondas de *boosting*. Más estimadores aumentan la cantidad de modelos más débiles que se utilizan para la estimación, el rendimiento del modelo y el tiempo de ejecución. El que el mejor valor haya resultado tan cerca de $1000$ sugiere que no se trataba de un rango no razonable.
- `max_depth` controla la máxima profundidad de cada árbol. Valores más altos hacen del modelo uno más complejo y propenso a sobreajustar. El valor por defecto es 6. Un valor de 10 parece suficientemente razonable.
- `max_leaves` se refiere a la cantidad de nodos que se agregan con cada árbol. Por defecto, este valor es cero. Valores más grandes aumentan la complejidad del modelo y el riesgo de sobreajuste. Optuna encontró que un valor de $35$ es óptimo (incluso en un rango que contiene al valor por defecto), sugiriendo que un modelo más complejo es deseable.
- `min_child_weight` establece el peso mínimo que debe tener una hoja para que se produzca una división. Reducir este número permite evitar que la computación se realice con sets de datos demasiado específicos, llevando a sobreajuste. El rango de valores propuesto es considerablemente pequeño ante la cantidad de datos disponibles, lo que puede llevar a una elección de parámetros que sobreajuste.
- `reg_alpha` establece la regularización L1 a todos los pesos. Un valor de 1 reduciría el impacto de todos los pesos. Su valor por defecto es 1, y su rango es $[0,\inf]$. Elegir valores en $[0,1]$ significa probar con regularizaciones relativamente débiles.
- `reg_lambda` es equivalente a `reg_alpha` para regularización L2. Al igual que la anterior, números más grandes llevan a modelos más conservadores.
- `min_frequency`especifica la mínima frecuencia o cantidad que una categoría debiese tener para ser considerada infrecuente. Valores enteros representan valores absolutos, mientras que decimales representan porcentajes sobre la población total. Un valor de $0.08$ indica que la categoría debe ser más común que el $8\%$ por sobre el total para que se le asigne una columna. Este valor optimizado podría ser demasiado pequeño, al dejar fuera ciudades que aún sin tanta frecuencia entregan información importante para la toma de decisiones.

**3.5.**

In [20]:
with open(f'./pickle_3.pkl', 'wb') as pkl_file:
    pickle.dump(study.user_attrs['best_pipeline'], pkl_file)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

**Prunning** es una técnica utilizada para detener tempranamente los ensayos de hiperparámetros que no muestran promesas de mejora. En lugar de esperar a que cada ensayo complete su entrenamiento, evalúa el rendimiento intermedio y decide si continuar o detener el ensayo. Esta técnica es especialmente útil en la optimización de hiperparámetros, donde el tiempo y los recursos computacionales son limitados.

La implementación de prunning en el entrenamiento de modelos debería tener un impacto significativo. Al detener los ensayos no prometedores de manera temprana, se pueden reasignar los recursos a ensayos más prometedores, acelerando el proceso de optimización. Esto no solo reduce el tiempo de entrenamiento, sino que también aumenta la eficiencia del proceso, permitiendo explorar un mayor número de configuraciones de hiperparámetros en el mismo período.

In [21]:
#import optuna
#from optuna.integration import XGBoostPruningCallback
#from xgboost import XGBRegressor
#from sklearn.pipeline import Pipeline
#from sklearn.compose import ColumnTransformer
#from sklearn.preprocessing import OneHotEncoder, StandardScaler
#from sklearn.metrics import mean_absolute_error
#import pickle
#
#optuna.logging.set_verbosity(optuna.logging.WARNING)
#
#def objective(trial):
#    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
#    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
#    max_depth = trial.suggest_int('max_depth', 3, 10)
#    max_leaves = trial.suggest_int('max_leaves', 0, 100)
#    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
#    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
#    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)
#    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
#
#    col_transformer = ColumnTransformer(
#        transformers=[
#            ('date', date_transformer, ['date']),
#            ('onehot', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), ['city', 'shop', 'brand', 'container', 'capacity']),
#            ('scaler', StandardScaler(), ['lat', 'long', 'pop', 'price'])
#        ],
#        remainder='passthrough'
#    )
#
#    pipe_xgbreg = Pipeline([
#        ('preprocessor', col_transformer),
#        ('regressor', XGBRegressor(
#            learning_rate=learning_rate,
#            n_estimators=n_estimators,
#            max_depth=max_depth,
#            max_leaves=max_leaves,
#            min_child_weight=min_child_weight,
#            reg_alpha=reg_alpha,
#            reg_lambda=reg_lambda,
#            enable_categorical=True,
#            random_state=42,
#            eval_metric="mae"
#        ))
#    ])
#
#    pruning_callback = XGBoostPruningCallback(trial, "validation_0-mae")
#
#    pipe_xgbreg.fit(
#        X_train, y_train,
#        regressor__eval_set=[(X_dev, y_dev)],
#        regressor__early_stopping_rounds=50,
#        regressor__callbacks=[pruning_callback],
#        regressor__verbose=False
#    )
#
#    y_pred = pipe_xgbreg.predict(X_dev)
#    error = mean_absolute_error(y_dev, y_pred)
#
#    trial.set_user_attr("best_model", pipe_xgbreg)
#
#    return error
#
#study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction="minimize")
#study.optimize(objective, timeout=300, show_progress_bar=True)
#
#print(f"Total de trials realizados: {len(study.trials)}")
#print(f"Mejor MAE: {study.best_value}")
#print(f"Mejores hiperparámetros: {study.best_params}")
#
#best_model = study.best_trial.user_attrs["best_model"]
#
#with open('best_xgboost_model_pruned.pkl', 'wb') as f:
#    pickle.dump(best_model, f)
#

In [22]:
#study = optuna.create_study(sampler=TPESampler(), direction="minimize")

#study.optimize(objective, timeout=300, show_progress_bar=True)

In [23]:
# No puedo aaaa

## 5. Visualizaciones (5 puntos)

Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [24]:
optuna.visualization.plot_optimization_history(study)

In [25]:
optuna.visualization.plot_parallel_coordinate(study)

In [26]:
optuna.visualization.plot_param_importances(study)

¿Desde qué trial se empiezan a observar mejoras notables en sus resultados?

Las mejoras significativas en el valor objetivo comienzan a ser notables alrededor del trial número 20. Aunque en los primeros intentos hay fluctuaciones grandes, a partir de ese punto los valores se estabilizan y tienden a acercarse al valor óptimo, lo que indica que las configuraciones de hiperparámetros empiezan a ser más eficientes.

¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?

En el gráfico de coordenadas paralelas, se observa que el valor objetivo está altamente influenciado por ciertas combinaciones de hiperparámetros. Se destaca que un learning rate elevado, junto con un número de n_estimators cercano a los valores superiores y valores específicos de max_depth y min_child_weight, tienden a dar mejores resultados. Además, algunos hiperparámetros, como reg_lambda, presentan variaciones significativas, mientras que otros no parecen tener un impacto tan pronunciado.

¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

Según el gráfico de importancia de hiperparámetros, el más influyente es claramente el learning_rate, con una importancia destacada de 0.78. Otros hiperparámetros relevantes incluyen min_child_weight y reg_lambda, aunque con una menor relevancia. En general, la tasa de aprendizaje parece ser la clave principal para mejorar el rendimiento del modelo.

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [29]:
mae_results = {
    'Modelo': ['Baseline (Dummy)', 'XGBoost', 'XGBoost con Constraints', 'XGBoost con Optuna'],
    'MAE en Validación': [mae_baseline, mae_xgboost, mae_xgboost_r, '1970.617']
}

mae_table = pd.DataFrame(mae_results)
print(mae_table)

                    Modelo MAE en Validación
0         Baseline (Dummy)      13298.497767
1                  XGBoost       2400.277512
2  XGBoost con Constraints       2602.156124
3       XGBoost con Optuna          1970.617


De la tabla anterior, el modelo que obtuvo el mejor rendimiento es XGBoost con Optuna, con un MAE de 1970.62 en el conjunto de validación. Esto indica que la optimización de hiperparámetros con Optuna mejoró el desempeño del modelo en comparación con las otras versiones. Posiblemente con prunning sea mejor.

In [35]:
with open('pickle_3.pkl', 'rb') as file:
    best_model = pickle.load(file)

best_model.fit(X_train, y_train)

y_test_pred = best_model.predict(X_test)

test_mae = mean_absolute_error(y_test, y_test_pred)
print(f"MAE en el conjunto de test: {test_mae}")


MAE en el conjunto de test: 2009.8275725745643


c:\Users\rodri\AppData\Local\Programs\Python\Python312\Lib\site-packages\xgboost\core.py:158: UserWarning:

[00:18:26] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the booster.
- Set the device for booster before call to inplace_predict.

This warning will only be shown once.




El modelo optimizado con Optuna logra un MAE de 2009.83 en el conjunto de test, apenas superior al 1970.62 obtenido en validación. Esta ligera diferencia refleja la posibilidad de factores como sobreajuste o variaciones en la distribución de datos. Sin embargo, el desempeño es sólido y consistente, indicando que el modelo generaliza bien pese a leves ajustes necesarios para adaptarse a las características únicas del conjunto de test. El modelo se comporta robustamente y confirma la mejora alcanzada en la optimización de sus hiperparámetros.

Aunque faltó el optuna con prunning :(

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.